In [78]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
import time
from math import log10, floor
import os

from relational_rnn_general import RelationalMemory

In [96]:
# Define variables
n_vars = 3
var_ids = list(range(n_vars))
var_names = ['var' + str(i) for i in var_ids]
var_weights = [0.1, 0.6, 0.3] # variable distribution of mock data
n_time_steps = 6
n_individuals = 1000

noise_length = 2

In [3]:
# Helper function(s)

# round a number to n significant digits
def round_to_n(x, n = 2):
    return round(x, -int(floor(log10(abs(x)))) + (n - 1)) if x != 0 else 0

# visualize the output of the generator
def visualize_output(generator, z, n = 2):
    p = generator(z).view(n_time_steps, n_vars)
    p.shape
    for t in range(p.shape[0]):
        tmp = []
        for f in range(p.shape[1]):
            tmp.append(round_to_n(p[t,f], n))
        print(tmp)

In [14]:
# Generate mock data

events = []

start_time = time.time()

for indv in range(n_individuals):
    tmp = []
    for t in range(n_time_steps):
        if t > 0 and tmp[t - 1] == 'var2':
            weights = [0.7, 0.2, 0.1]
            var = np.random.choice(var_names, p=weights)
        else:
            var = np.random.choice(var_names, p=var_weights)
        tmp.append(var)
    events.append(tmp)
        
print('time taken:', round_to_n(time.time() - start_time), 'seconds')

for i in range(10):
    print(events[i])

In [44]:
vars_to_indices = dict([(v, i) for i, v in enumerate(var_names)])
print(vars_to_indices)
data = torch.tensor([[vars_to_indices[e] for e in event] for event in events])
data[:10]

{'var0': 0, 'var1': 1, 'var2': 2}


tensor([[0, 1, 1, 1, 2, 0],
        [0, 1, 2, 0, 1, 1],
        [1, 1, 2, 0, 2, 0],
        [1, 1, 1, 1, 1, 2],
        [2, 0, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1],
        [2, 1, 2, 0, 2, 0],
        [2, 0, 2, 0, 0, 0],
        [1, 1, 1, 2, 0, 1],
        [1, 1, 1, 2, 0, 1]])

In [98]:
# Define the generator
class Generator(nn.Module):
    def __init__(self, mem_slots, head_size, num_heads, temperature, vocab_size, noise_length):
        super(Generator, self).__init__()
        #self.embed = nn.Embedding(vocab_size, embed_vector_size)
        self.relational_memory = RelationalMemory(mem_slots, head_size, noise_length, num_heads)
        
        self.temperature = temperature
        
        self.output_layer = nn.Linear(self.relational_memory.mem_size * self.relational_memory.mem_slots, vocab_size)
        
    def initial_state(self, batch_size):
        return self.relational_memory.initial_state(batch_size)
    
    def gumbel_softmax(self, output, temperature, eps = 1e-10):
        u = torch.rand(output.shape)
        g = -torch.log(-torch.log(u + eps) + eps)
        result = torch.mul(output.add(g), temperature)
        result = F.softmax(result, dim=1)
        return result
    
    def forward(self, inputs, memory):
        #embed = self.embed(inputs)
        output, memory = self.relational_memory(inputs, memory)
        output = self.output_layer(output)
        output = F.softmax(output, dim=1)
        output = self.gumbel_softmax(output, self.temperature)
        return output, memory

G = Generator(mem_slots = 4, head_size = 2, num_heads = 3, temperature = 1, vocab_size = n_vars, noise_length = noise_length)
#print(G.state_dict())
memory = G.initial_state(batch_size = 1)

In [99]:
#inp = data[0, :].view(1, -1)
inp = torch.randn(1, n_time_steps, noise_length)
memory = G.initial_state(batch_size = 1)
print(inp)
print(memory)
logits, memory = G(inp, memory)
print(logits)
print(memory)

tensor([[[-0.6229, -0.6879],
         [-0.2634, -0.0263],
         [-0.2806, -0.8714],
         [ 1.2484,  0.0643],
         [-1.8433, -1.4589],
         [ 0.0105,  0.9897]]])
tensor([[[1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.]]])


AttributeError: 'Generator' object has no attribute 'embed'

In [95]:
test = torch.tensor([[1,2,3]])
print(test.shape)
embed = nn.Embedding(10, 4)
test_embed = embed(test)
print(test_embed)
print(test_embed.shape)
test_inp = test_embed[:, 0]
print(test_inp.shape)
test_inp = test_inp.view(test_inp.shape[0], -1)
print(test_inp.shape)
test_inp.unsqueeze(dim=1).shape



torch.Size([1, 3])
tensor([[[ 0.6941,  0.1010,  0.5416, -0.4480],
         [-0.2943,  1.1283,  0.1775,  0.6900],
         [ 0.2171,  0.4527, -0.7026, -0.4551]]], grad_fn=<EmbeddingBackward>)
torch.Size([1, 3, 4])
torch.Size([1, 4])
torch.Size([1, 4])


torch.Size([1, 1, 4])